Model for SUDOKU Digit Recognition


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler


Using TensorFlow backend.


Here we will first train the model on **MNIST data** then it will be trained on our data for sudoku digit recognition.

In [0]:
#Unzip the file train.csv
!unzip /content/data.zip

In [79]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
image = load_img('/content/rot/img001-00001.png')
array = img_to_array(image)
array.shape

(128, 128, 3)

In [72]:

img_width,img_height = 28,28
batch_size = 64
train_data_dir = '/content/data'
train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.1,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    validation_split=0.1)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    color_mode="grayscale",
    subset='training')
validation_generator = train_datagen.flow_from_directory(
    train_data_dir, # same directory as training data
    target_size=(img_width, img_height),
    color_mode="grayscale",
    batch_size=batch_size,
    subset='validation')

Found 8235 images belonging to 9 classes.
Found 909 images belonging to 9 classes.


In [0]:
model = Sequential()

model.add(Conv2D(32, kernel_size = 3, activation='relu', input_shape = (28, 28, 1)))
model.add(BatchNormalization())
model.add(Conv2D(32, kernel_size = 3, activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(32, kernel_size = 5, strides=2, padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Conv2D(64, kernel_size = 3, activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size = 3, activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size = 5, strides=2, padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dense(9, activation='softmax'))

# COMPILE WITH ADAM OPTIMIZER AND CROSS ENTROPY COST
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [80]:
#Training the model
history = model.fit_generator(train_generator,steps_per_epoch=int(train_generator.samples/64),
                              epochs=10,validation_data=validation_generator,
                              validation_steps=int(validation_generator.samples/64))

Epoch 1/10
128/128 [==============================] - 10s 77ms/step - loss: 0.8365 - accuracy: 0.7299 - val_loss: 1.7704 - val_accuracy: 0.3549
Epoch 2/10
128/128 [==============================] - 9s 68ms/step - loss: 0.2194 - accuracy: 0.9264 - val_loss: 3.2008 - val_accuracy: 0.2095
Epoch 3/10
128/128 [==============================] - 9s 68ms/step - loss: 0.1436 - accuracy: 0.9498 - val_loss: 0.6922 - val_accuracy: 0.7077
Epoch 4/10
128/128 [==============================] - 9s 68ms/step - loss: 0.1230 - accuracy: 0.9583 - val_loss: 0.0033 - val_accuracy: 0.9893
Epoch 5/10
128/128 [==============================] - 9s 68ms/step - loss: 0.1033 - accuracy: 0.9648 - val_loss: 0.0373 - val_accuracy: 0.9834
Epoch 6/10
128/128 [==============================] - 9s 69ms/step - loss: 0.0933 - accuracy: 0.9687 - val_loss: 0.0122 - val_accuracy: 0.9988
Epoch 7/10
128/128 [==============================] - 9s 69ms/step - loss: 0.0733 - accuracy: 0.9745 - val_loss: 0.0785 - val_accuracy: 0.938

In [85]:
import cv2
image = cv2.imread("/content/data/2/img003-00008.png")
image1 = cv2.imread("/content/data/1/img002-00007.png")
 
image = cv2.resize(image,(28,28),interpolation=cv2.INTER_AREA)
image1 = cv2.resize(image1,(28,28),interpolation=cv2.INTER_AREA)
image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
image1 = cv2.cvtColor(image1,cv2.COLOR_BGR2GRAY)


preds = model.predict([image.reshape(-1,28,28,1)])
for cls in train_generator.class_indices:
    print(""+str(cls)+": "+str(preds[0][train_generator.class_indices[cls]]))

1: 0.0
2: 1.0
3: 0.0
4: 0.0
5: 0.0
6: 0.0
7: 0.0
8: 0.0
9: 0.0


In [0]:
model.save('digit.model')